In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
#import pandas_datareader as data_reader
import yfinance as yf

from tqdm import tqdm_notebook, tqdm
from collections import deque

**RL Trader Network**

In [2]:
class AI_Trader():
    
    def __init__(self, state_size, action_space=3, model_name="AITrader"): 

        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(maxlen=2000)
        self.inventory = []
        self.model_name = model_name

        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995

        self.model = self.model_builder()

    def model_builder(self):

        model = tf.keras.models.Sequential()

        model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=self.state_size))

        model.add(tf.keras.layers.Dense(units=64, activation="relu"))

        model.add(tf.keras.layers.Dense(units=128, activation="relu"))

        model.add(tf.keras.layers.Dense(units=self.action_space, activation="linear"))

        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

        return model

    def trade(self, state):

        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
    
        actions = self.model.predict(state)
        return np.argmax(actions[0])

    def batch_train(self, batch_size):
    
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])
    
        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

            target = self.model.predict(state)
            target[0][action] = reward

            self.model.fit(state, target, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay


**Dataset Preprocessing**

In [3]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [4]:
def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))
  

In [5]:
def dataset_loader(stock_name, start_date, end_date):
    
    #dataset = data_reader.DataReader(stock_name, data_source="yahoo")
    dataset = yf.Ticker(stock_name)
    dataset = dataset.history(start=start_date, end=end_date, interval='1d')
  
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]

    close = dataset["Close"]
  
    return close

In [6]:
def state_creator(data, timestep, window_size):
    
    starting_id = timestep - window_size + 1

    if starting_id >= 0:
        windowed_data = data[starting_id:timestep + 1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
  
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))

    return np.array([state])


**Training Loop**

In [7]:
def start_training(stock_name, start, end): # trader, window_size, episodes, batch_size):
    
    results = pd.DataFrame(columns=['Episode', 'Date', 'Stock', 'Buy', 'Sell', 'Profit'])
    
    data = dataset_loader(stock_name, start, end)

    for episode in range(1, episodes + 1):
        print(f"Episode: {episode}/{episodes}")

        state = state_creator(data, 0, window_size + 1)

        total_profit = 0
        trader.inventory = []
        data_samples = len(data) - 1

        for t in tqdm(range(data_samples)):
            
            action = trader.trade(state)
            next_state = state_creator(data, t+1, window_size + 1)
            reward = 0

            if action == 1: # Buying
                trader.inventory.append(data[t])
                print(f"AI Trader bought: {stocks_price_format(data[t])}")
                
                results = results.append({'Episode':episode, 'Date':data.index[t], 'Stock':stock_name, 
                                          'Buy':data[t], 'Sell':0, 'Profit':0}, ignore_index=True)


            elif action == 2 and len(trader.inventory) > 0: # Selling
                buy_price = trader.inventory.pop(0)
                
                profit = data[t] - buy_price
                reward = max(profit, 0)
                total_profit += data[t] - buy_price
                print(f"AI Trader sold on {data.index[t]} for: {stocks_price_format(data[t])} Profit: {stocks_price_format(profit)}" )
                
                results = results.append({'Episode':episode, 'Date':data.index[t], 'Stock':stock_name, 
                                          'Buy':0, 'Sell':data[t], 'Profit':profit}, ignore_index=True)

            if t == data_samples - 1:
                done = True
            else:
                done = False

            trader.memory.append((state, action, reward, next_state, done))

            state = next_state

            if done:
                print("####################")
                print(f"TOTAL PROFIT: {total_profit}")
                print("####################")

            if len(trader.memory) > batch_size:
                trader.batch_train(batch_size)

    return results


In [8]:
#Initialize values
stock_names = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL', 'TSLA'] #FB AMZN AAPL NFLX GOOGL TSLA]
start = '2020-01-01'
end = '2021-12-01'

#Set training parameters
window_size = 10
episodes = 2
batch_size = 32
overall_results = pd.DataFrame()

for stock in stock_names:
    trader = AI_Trader(window_size)
    stock_results = start_training(stock, start, end)
    overall_results = overall_results.append(stock_results, ignore_index=True)
    
print('Results are Saved')
overall_results.to_csv('faang_stock_results.csv')

  0%|                                                                                          | 0/482 [00:00<?, ?it/s]

Episode: 1/2
AI Trader bought: $ 213.059998
AI Trader sold on 2020-01-08 00:00:00 for: $ 215.220001 Profit: $ 2.160004
AI Trader bought: $ 221.910004
AI Trader bought: $ 219.059998
AI Trader bought: $ 221.149994
AI Trader sold on 2020-01-16 00:00:00 for: $ 221.770004 Profit: - $ 0.139999
AI Trader bought: $ 222.139999
AI Trader bought: $ 221.440002
AI Trader bought: $ 221.320007
AI Trader sold on 2020-01-23 00:00:00 for: $ 219.759995 Profit: $ 0.699997
AI Trader bought: $ 214.869995
AI Trader bought: $ 217.789993
AI Trader bought: $ 209.529999
AI Trader sold on 2020-02-03 00:00:00 for: $ 204.190002 Profit: - $ 16.959991
AI Trader sold on 2020-02-05 00:00:00 for: $ 210.110001 Profit: - $ 12.029999
AI Trader sold on 2020-02-06 00:00:00 for: $ 210.850006 Profit: - $ 10.589996
AI Trader sold on 2020-02-12 00:00:00 for: $ 210.759995 Profit: - $ 10.560013
AI Trader sold on 2020-02-13 00:00:00 for: $ 213.139999 Profit: - $ 1.729996
AI Trader bought: $ 214.179993
AI Trader bought: $ 217.800003

  7%|█████▌                                                                           | 33/482 [00:03<00:43, 10.36it/s]

AI Trader bought: $ 214.580002
AI Trader bought: $ 210.179993


  7%|█████▉                                                                           | 35/482 [00:08<02:16,  3.28it/s]

...

AI Trader sold on 2021-11-26 00:00:00 for: $ 1081.920044 Profit: $ 63.490051


100%|███████████████████████████████████████████████████████████████████████████████▊| 481/482 [21:25<00:02,  2.68s/it]

AI Trader sold on 2021-11-29 00:00:00 for: $ 1136.989990 Profit: $ 99.130005
####################
TOTAL PROFIT: 7731.756080627441
####################


100%|████████████████████████████████████████████████████████████████████████████████| 482/482 [21:28<00:00,  2.67s/it]

Results are Saved
